In [38]:
import pandas as pd
import sys
import os

from datetime import datetime

In [9]:
columns_name = ['ticker', 'per', 'date', 'time', 'close']

In [167]:
def change_price(s):
    splited = s.split('.')
    part1 = int(''.join(splited[:-1]))
    part2 = float('0.' + splited[-1])
    res = part1 + part2
    return res


def read_files():
    columns_name = ['ticker', 'per', 'date', 'time', 'close']
    dfs = {}
    for f_name in os.listdir('moscow_exchange'):
        path = 'moscow_exchange/' + f_name
        df = pd.read_csv(path, names=columns_name, skiprows=1)
        ticker = df['ticker'][0]
        dfs[ticker] = df.drop(['ticker', 'per', 'time'], axis=1)
        
        if dfs[ticker]['close'].dtype == 'O':
            dfs[ticker]['close'] = dfs[ticker]['close'].apply(change_price)
        
    return dfs


def filter_companies(dfs_first_dates, date):
    filtered_list = []
    for k, v in dfs_first_dates.items():
        if v < date:
            filtered_list.append(k)
    return filtered_list


def make_df(dfs, date_to_filter):
    old_companies = filter_companies(dfs_first_dates, date_to_filter)
    old_companies_dates_dfs = {}
    for comp in old_companies:
        old_companies_dates_dfs[comp] = set(dfs[comp]['date'])

    common_dates = old_companies_dates_dfs[next(iter(old_companies_dates_dfs))]
    for k, v in old_companies_dates_dfs.items():
        common_dates.intersection_update(v)
    
    dfs_old = []
    for comp in old_companies:
        df = dfs[comp][dfs[comp]['date'].isin(common_dates)]
        df.drop('date', inplace=True, axis=1)
        df.columns = [comp]
        dfs_old.append(df.reset_index(drop=True))
    res_df = pd.concat(dfs_old, axis=1)
    return res_df

In [173]:
dfs = read_files()

In [67]:
dfs_first_dates = {}
for k, v in dfs.items():
    first_date = datetime.date(datetime.strptime(str(v['date'].iloc[0]), "%Y%m%d"))
    dfs_first_dates[k] = first_date

In [219]:
df_main_stocks = make_df(dfs, datetime.date(datetime(2006, 1, 1)))

/home/yury/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [233]:
df_main_stocks.head()

,SBER,TRNFP,LKOH,VSMO,SNGS,AFLT,KMAZ,CHMF,RTKM,MTSS,SVAV,TATN,GMKN
0,19180.00,26500.00,1053.97,3300.0,21.345,42.29,13.75,208.00,57.49,191.0,410.0,52.912,1736.50
1,19295.00,28501.00,1098.70,3425.0,22.098,42.20,15.73,216.00,57.70,194.0,412.0,55.397,1745.85
2,20308.00,29500.00,1117.39,3500.0,22.483,42.50,15.45,225.00,58.68,198.0,413.0,57.984,1805.22
3,20885.00,29280.00,1133.51,3560.0,22.700,43.10,15.66,229.99,59.87,205.0,419.0,57.512,1844.88
4,20991.03,28905.02,1139.50,3545.0,22.670,43.00,15.87,230.00,60.00,207.5,420.0,56.878,1849.99


In [230]:
df_new_stocks = make_df(dfs, datetime.date(datetime(2014, 6, 5)))

/home/yury/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [234]:
df_new_stocks.head()

,URKA,MGNT,NLMK,AFKS,SBER,MVID,TRNFP,LKOH,VSMO,AKRN,...,LSRG,RTKM,NVTK,VTBR,MTSS,SVAV,GAZP,TATN,GMKN,TRMK
0,157.01,8970.0,48.33,43.400,87.81,260.02,77663.0,2047.0,8300.0,1369.9,...,613.0,86.74,382.15,0.05112,290.49,617.0,144.04,210.47,6898.0,81.39
1,154.50,8928.8,47.87,43.750,87.43,259.81,76500.0,2035.0,8200.0,1368.3,...,615.1,84.71,392.56,0.04844,297.00,645.5,142.75,208.79,6882.0,81.40
2,156.99,8980.0,49.27,44.290,89.00,258.99,77870.0,2030.6,8250.0,1234.0,...,623.9,86.16,403.17,0.04689,303.30,647.0,143.97,209.55,6975.0,85.58
3,154.05,8949.0,49.42,44.448,88.89,254.45,77050.0,2025.0,8020.0,1228.0,...,620.8,87.69,405.93,0.04731,297.55,648.0,144.40,207.63,7029.0,87.08
4,157.18,9100.0,48.60,45.499,88.99,256.00,77055.0,2061.4,8096.4,1196.0,...,627.5,88.80,401.37,0.04700,296.63,593.0,144.75,211.00,6900.0,89.69


In [214]:
df_main_stocks.to_csv('data/moscow_main.csv')

In [232]:
df_new_stocks.to_csv('data/moscow_new.csv')

In [231]:
df_new_stocks.keys()

Index(['URKA', 'MGNT', 'NLMK', 'AFKS', 'SBER', 'MVID', 'TRNFP', 'LKOH', 'VSMO',
       'AKRN', 'SNGS', 'MAGN', 'AFLT', 'MSTT', 'KMAZ', 'CHMF', 'POLY', 'MOEX',
       'PIKK', 'YNDX', 'ALRS', 'NMTP', 'MFON', 'ROSN', 'IRAO', 'BANE', 'HYDR',
       'RUALR', 'BSPB', 'LSRG', 'RTKM', 'NVTK', 'VTBR', 'MTSS', 'SVAV', 'GAZP',
       'TATN', 'GMKN', 'TRMK'],
      dtype='object')